Example Script Associated with:
# Speed of sound of pure water to 700 MPa and an equation of state to 2300 MPa
## by O. Bollengier, J.M. Brown, and G. Shaw

The following demonstrates the use of a "Local Basis Function" (LBF) representation for the Gibbs energy equation of state of water.   

See [the LocalBasisFunction repository](https://github.com/jmichaelb/LocalBasisFunction/tree/master/Python) for installation requirements.

Units: MPa for pressure, K for temperature, S.I. units for density, Cp, and G (kg, J, m)

Each individual code block below can be executed by clicking the play icon to the left of the block, or by using the Run button at the top. Comments in the code are preceded by a hash.

### 1. Load data into the environment and describe the content

The LBF is stored as a struct in a MATLAB data file, with the range of coverage incorporated into the name.  This struct is loaded into the Python environment as a dictionary, using only those elements needed for evaluation of the spline.  The contents of the spline are then shown. 

Keys in the dictionary match the standard element names used by MATLAB representations of tensor splines. See [MATLAB documentation](https://www.mathworks.com/help/curvefit/multivariate-tensor-product-splines.html) for more information.  

In [134]:
import numpy as np

from mlbspline import load
from lbftd import statevars, loadGibbs as lg, evalGibbs as eg

import matplotlib.pyplot as plt
from matplotlib import rcParams
from mpl_toolkits.mplot3d import Axes3D

G_H2O_2GPa_500K = load.loadSpline('../G_H2O_2GPa_500K.mat')
G_H2O_2GPa_500K

{'form': 'B-',
 'knots': array([array([   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,   15.        ,   20.        ,
          25.        ,   30.        ,   35.        ,   40.        ,
          45.        ,   50.        ,   55.        ,   60.        ,
          65.        ,   70.        ,   75.        ,   80.        ,
          85.        ,   90.        ,   95.09172249,  100.33476036,
         105.79195946,  111.52959614,  117.61756454,  124.03785154,
         130.80859712,  137.94893146,  145.47902897,  153.42016533,
         161.79477754,  170.62652738,  179.94036821,  189.76261551,
         200.12102122,  211.04485216,  222.56497269,  234.71393195,
         247.52605581,  261.03754386,  275.28657167,  290.31339869,
         306.16048195,  322.87259607,  340.49695973,  359.08336909,
         378.6843385 ,  399.35524887,  421.15450412,  444.14369622,
         468.38777922,  493.95525276,  520.91835557,  549.35326967,
         579.3403

### Extract thermodynamic properties from the LBF representation

The thermodynamic variables currently supported for pure substances are listed below.  For each tuple, the second value is descriptive only; the first value is what would be passed to the evaluation function to limit which variables are calculated. 

In [135]:
[(sv.name, sv.calcFn.__name__[4:], ) for sv in statevars.statevars if not sv.reqM]

[('G', 'GibbsEnergy'),
 ('rho', 'Density'),
 ('vel', 'SoundSpeed'),
 ('Cp', 'IsobaricSpecificHeat'),
 ('Cv', 'IsochoricSpecificHeat'),
 ('alpha', 'ThermalExpansivity'),
 ('U', 'InternalEnergy'),
 ('H', 'Enthalpy'),
 ('S', 'Entropy'),
 ('Kt', 'IsothermalBulkModulus'),
 ('Kp', 'IsothermalBulkModulusWrtPressure'),
 ('Ks', 'IsotropicBulkModulus'),
 ('mus', 'SoluteChemicalPotential'),
 ('Vm', 'PartialMolarVolume'),
 ('Cpm', 'PartialMolarHeatCapacity'),
 ('V', 'Volume')]

#### Calculating thermodynamic properties for scattered data points

To calculate thermodynamic variables (tdvs) for scattered pressures and temperatures, basic syntax is

    out_scatter = eg.evalSolutionGibbsGrid(gibbsSp, PTM, *tdvSpec)
    
where 
* ```gibbsSp``` is a Gibbs energy LBF with dimensions pressure (MPa) and temperature (K), in the order defined by ```statevars.iP``` and ```statevars.iT```, respectively. 
* ```PTM``` is a Numpy ndarray of tuples with the points at which ```gibbsSp``` should be evaluated. The number of elements in each tuple must be same as in the spline (```PTM[0].size == gibbsSp['number'].size```).  Each tuple gives the pressure (P) and temperature (T) in the same order and units as in the ```gibbsSp``` parameter.  No sorting is required.
* ```tdvSpec``` is an optional list of the tdvs to be calculated.  If no values are provided, this function will calculate all supported tdvs as listed above.  Requesting an unsupported value will result in an error; note that values are case-sensitive.  See [the LocalBasisFunction source code repository](https://github.com/jmichaelb/LocalBasisFunction/tree/master/Python) for units of each tdv.

The output of the function call is an object with the requested tdvs as named properties matching those requested in the ```*tdvSpec``` parameter (all supported values if no list of values provided). The object also includes a ```PTM``` property that contains a copy of the input ```PTM``` parameter.

The following code block calculates all supported properties at STP.

In [136]:
print({'pressure_index': statevars.iP, 'temp_index': statevars.iT})
PTM = np.empty((1,), np.object)
PTM[0] = (0.1, 300)  # pressure = 0.1 MPa, temperature = 300 K
out_single = eg.evalSolutionGibbsScatter(G_H2O_2GPa_500K, PTM)
vars(out_single) # Note: this does not include the PTM attribute.

{'pressure_index': 0, 'temp_index': 1}


{'G': array([-5267.28216317]),
 'rho': array([996.5563895]),
 'vel': array([1501.700531]),
 'Cp': array([4180.59628951]),
 'Cv': array([4130.12907247]),
 'alpha': array([0.00027479]),
 'U': array([112549.08139637]),
 'H': array([-5367.62771417]),
 'S': array([393.05569704]),
 'Kt': array([2220.20941558]),
 'Kp': array([5.57619587]),
 'Ks': array([2247.33878332]),
 'V': array([0.00100346])}

For comparison, here is the same information calculated from an LBF based on IAPWS-95.  See [doi:10.1016/j.fluid.2018.02.001](https://doi.org/10.1016/j.fluid.2018.02.001) for details on the differences between this and the actual IAPWS 95 values.

In [137]:
iapws_sp = load.loadSpline('./IAPWS_sp_strct.mat')
iapws_out = eg.evalSolutionGibbsScatter(iapws_sp, PTM)
vars(iapws_out)

{'G': array([-5265.19521713]),
 'rho': array([996.5563392]),
 'vel': array([1501.54072197]),
 'Cp': array([4180.75236869]),
 'Cv': array([4130.2900946]),
 'alpha': array([0.0002748]),
 'U': array([112556.37101179]),
 'H': array([-5365.54077318]),
 'S': array([393.07303928]),
 'Kt': array([2219.74045413]),
 'Kp': array([5.70772829]),
 'Ks': array([2246.86037759]),
 'V': array([0.00100346])}

To calculate scattered data for several points, here we calculate density for five states:

| Pressure (MPa) | Temperature (K) |
| --------------:| ---------------:|
|       441.0858 |        313.9500 |
|       478.7415 |        313.9600 |
|       444.8285 |        313.7800 |
|       459.1574 |        313.8000 |
|      485.4930  |        313.8200 |

Each tdv property in the output contains an array with indices corresponding to those in the input ```PTM``` parameter.  

In [138]:
PTM = np.empty((5,), np.object)
PTM[0] = (441.0858, 313.9500)
PTM[1] = (478.7415, 313.9600)
PTM[2] = (444.8285, 313.7800)
PTM[3] = (459.1574, 313.8000)
PTM[4] = (485.4930, 313.8200)
out_scatter = eg.evalSolutionGibbsScatter(G_H2O_2GPa_500K, PTM, 'rho') # calculate just one of the tdvs
out_scatter.rho

array([1128.21830415, 1136.65424508, 1129.1580671 , 1132.38514694,
       1138.20701159])

#### Calculating and plotting thermodynamic properties for gridded data points

To calculate thermodynamic variables for a grid of pressures and temperatures, basic syntax is

    output = eg.evalSolutionGibbsGrid(gibbsSp, PTM, *tdvSpec)

where the ```gibbsSp``` and ```tdvSpec``` parameters are the same as in ```evalSolutionGibbsScatter```, but the ```PTM``` parameter has a different format.  Here, ```PTM``` is a Numpy ndarray of ndarrays with the conditions at which ```gibbsSp``` should be evaluated. The number of dimensions must be same as in the spline (```PTM.size == gibbsSp['number'].size```), with the first inner array giving a list of pressures and the second a list of temperatures at which tdvs should be calculated, as dictated by ```statevars.iP``` and ```statevars.iT```.  Both inner arrays must be sorted from low to high values, and be expressed in the units indicated in the ```gibbsSp``` parameter. 

The output of the function call is an object with the requested tdvs as named properties matching those requested in the ```*tdvSpec``` parameter.  The output also includes a ```PTM``` property so that the regime for each tdv is easily passed along with the calculated values.  

The call below calculates a subset of supported tdvs: 
* density (rho)
* isobaric specific heat (Cp)
* isothermal bulk modulus (Kt)
* thermal expansivity (alpha)

Note that only requested tdvs will be included in the output.

In [139]:
P = np.arange(0.1, 1000.2, 10)  # pressures from 0.1 - 1000.1 MPa
T = np.arange(240, 501, 2)      # temperatures from 240 - 500 K
out_grid = eg.evalSolutionGibbsGrid(G_H2O_2GPa_500K, np.array([P, T]), 'rho', 'Cp', 'Kt', 'alpha')
vars(out_grid).keys()

dict_keys(['rho', 'Cp', 'Kt', 'alpha'])

Output values can be addressed using object/property syntax.  Each tdv is a matrix, with the first index corresponding to a pressure in ```out_grid.PTM[statevars.iP]``` and the second corresponding to a temperature in ```out_grid.PTM[statevars.iT]```.

In [140]:
pi = np.random.randint(0,out_grid.PTM[statevars.iP].size) # the index of one pressure at which tdvs were evaluated
ti = np.random.randint(0,out_grid.PTM[statevars.iT].size) # the index of one temperature at which tdvs were evaluated 
"P={:.4f} MPa, T={:.4f} K, density={:.1f} kg/m^3".format(
    out_grid.PTM[statevars.iP][pi], out_grid.PTM[statevars.iT][ti], out_grid.rho[pi,ti])


'P=630.1000 MPa, T=300.0000 K, density=1175.2 kg/m^3'